# Base de datos. Tratamiento y limpieza 🕵🏻

En este jupyter se encuentra todo el proceso de limpieza y selección, de los datos descargados de kaggle "COVID-19 | DATASET"

*Consideraciones*

Para limpiar el dataframe original y obtener como resultado final "covid_data" se ha tenido en cuenta la siguiente consideración la cual afectará al número de datos seleccionados para realizar más tarde su pertinente análisis. "El estudio sólo recogerá los siguientes países: España, Francia, Italia, Grecia, Turquia, USA, China, India, Mexico, Nueva Zelanda y Reino Unido".

## Índice 📎

1. Importación de librerías y funciones
2. Importación del dataset
3. Exploración del dataset
4. Limpieza de datos
5. Exportación del dataset final

## 1. Importación de librerias y funciones 📚

In [1]:
import pandas as pd
import numpy as np
import re

Otras funciones

In [3]:
from iteration_utilities import duplicates
from iteration_utilities import unique_everseen
import Funciones as fu

## 2. Importación del dataset 📖

In [3]:
data = pd.read_csv("./covid_cases.csv")
data.columns =  data.columns.str.rstrip() #Elimina los espacios al final de cada título en las columnas

In [4]:
data.head()

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


## 3. Exploración del dataset 🔎

El primer paso será observar todas las columnas con el objetivo de identificar la información necesaria para nuestra investigación.

In [5]:
data.columns

Index(['Country/Region', 'Confirmed', 'Deaths', 'Recovered', 'Active',
       'New cases', 'New deaths', 'New recovered', 'Deaths / 100 Cases',
       'Recovered / 100 Cases', 'Deaths / 100 Recovered',
       'Confirmed last week', '1 week change', '1 week % increase',
       'WHO Region'],
      dtype='object')

En toda base de datos puede darse el caso de que existan valores nulos, información repetida o columnas que no reporten ningún tipo de información de valor o relevancia al caso de estudio. Para ello, procederemos a comprobar el número de NaNs.

*Se empleará la función isnull recogida en el archivo de rsc*

In [6]:
def isnull(x):
    return x.isnull().sum().sort_values(ascending=False)

In [7]:
isnull(data)

Country/Region            0
Confirmed                 0
Deaths                    0
Recovered                 0
Active                    0
New cases                 0
New deaths                0
New recovered             0
Deaths / 100 Cases        0
Recovered / 100 Cases     0
Deaths / 100 Recovered    0
Confirmed last week       0
1 week change             0
1 week % increase         0
WHO Region                0
dtype: int64

Como podemos observar no existen NaNs en nuestro dataset

Puesto que una de nuestras principales variables será el país que se encuentra en la columna "Country/Region" vamos a comprobar si hay datos duplicados, a fin de garantizar la fiabilidad del dataset.

*Para ello se empleará la función comprobacion_duplicados*

In [8]:
def comprobacion_duplicados (x):
    repetidos = list(unique_everseen(duplicates(x)))
    return len(repetidos)

In [9]:
country = data["Country/Region"]
print(f"La columna que indica el país tiene {comprobacion_duplicados(country)} datos duplicados")

La columna que indica el país tiene 0 datos duplicados


A continuación, analizaremos los tipos de datos que se nos facilitan. Es importante, para más tarde realizar las operaciones pertinentes. 

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Country/Region          187 non-null    object 
 1   Confirmed               187 non-null    int64  
 2   Deaths                  187 non-null    int64  
 3   Recovered               187 non-null    int64  
 4   Active                  187 non-null    int64  
 5   New cases               187 non-null    int64  
 6   New deaths              187 non-null    int64  
 7   New recovered           187 non-null    int64  
 8   Deaths / 100 Cases      187 non-null    float64
 9   Recovered / 100 Cases   187 non-null    float64
 10  Deaths / 100 Recovered  187 non-null    float64
 11  Confirmed last week     187 non-null    int64  
 12  1 week change           187 non-null    int64  
 13  1 week % increase       187 non-null    float64
 14  WHO Region              187 non-null    ob

Por último a propósito de tener una idea global antes de realizar el análisis y limpieza, obervaremos algunos datos numéricos.

In [11]:
data.describe()

,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase
count,1.870000e+02,187.000000,1.870000e+02,1.870000e+02,187.000000,187.000000,187.000000,187.000000,187.000000,187.00,1.870000e+02,187.000000,187.000000
mean,8.813094e+04,3497.518717,5.063148e+04,3.400194e+04,1222.957219,28.957219,933.812834,3.019519,64.820535,inf,7.868248e+04,9448.459893,13.606203
std,3.833187e+05,14100.002482,1.901882e+05,2.133262e+05,5710.374790,120.037173,4197.719635,3.454302,26.287694,NaN,3.382737e+05,47491.127684,24.509838
min,1.000000e+01,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,1.000000e+01,-47.000000,-3.840000
25%,1.114000e+03,18.500000,6.265000e+02,1.415000e+02,4.000000,0.000000,0.000000,0.945000,48.770000,1.45,1.051500e+03,49.000000,2.775000
50%,5.059000e+03,108.000000,2.815000e+03,1.600000e+03,49.000000,1.000000,22.000000,2.150000,71.320000,3.62,5.020000e+03,432.000000,6.890000
75%,4.046050e+04,734.000000,2.260600e+04,9.149000e+03,419.500000,6.000000,221.000000,3.875000,86.885000,6.44,3.708050e+04,3172.000000,16.855000
max,4.290259e+06,148011.000000,1.846641e+06,2.816444e+06,56336.000000,1076.000000,33728.000000,28.560000,100.000000,inf,3.834677e+06,455582.000000,226.320000


## 4. Limpieza de datos 🧹

Para nuestro estudio no será necesario contar con ciertas columnas por lo que procederemos a eliminarlas. Por ende no se tomarán en cuenta los datos referentes a "New cases", "New deaths", "New recovered", "Confirmed last week", "1 week change","1 week % increase" y "WHO Region". 

In [12]:
data2 = data.drop(["New cases", "New deaths", "New recovered", "Confirmed last week", "1 week change", "1 week % increase","WHO Region"], axis=1)
data2

,Country/Region,Confirmed,Deaths,Recovered,Active,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered
0,Afghanistan,36263,1269,25198,9796,3.50,69.49,5.04
1,Albania,4880,144,2745,1991,2.95,56.25,5.25
2,Algeria,27973,1163,18837,7973,4.16,67.34,6.17
3,Andorra,907,52,803,52,5.73,88.53,6.48
4,Angola,950,41,242,667,4.32,25.47,16.94
...,...,...,...,...,...,...,...,...
182,West Bank and Gaza,10621,78,3752,6791,0.73,35.33,2.08
183,Western Sahara,10,1,8,1,10.00,80.00,12.50
184,Yemen,1691,483,833,375,28.56,49.26,57.98
185,Zambia,4552,140,2815,1597,3.08,61.84,4.97


Para facilitar la legibilidad procederemos a renombrar las columnas

In [13]:
columnas = ["country","confirmed","deaths","recovered","active","%deaths","%recovered","%D/100R"]

In [14]:
data2.columns = columnas
data2

,country,confirmed,deaths,recovered,active,%deaths,%recovered,%D/100R
0,Afghanistan,36263,1269,25198,9796,3.50,69.49,5.04
1,Albania,4880,144,2745,1991,2.95,56.25,5.25
2,Algeria,27973,1163,18837,7973,4.16,67.34,6.17
3,Andorra,907,52,803,52,5.73,88.53,6.48
4,Angola,950,41,242,667,4.32,25.47,16.94
...,...,...,...,...,...,...,...,...
182,West Bank and Gaza,10621,78,3752,6791,0.73,35.33,2.08
183,Western Sahara,10,1,8,1,10.00,80.00,12.50
184,Yemen,1691,483,833,375,28.56,49.26,57.98
185,Zambia,4552,140,2815,1597,3.08,61.84,4.97


Procedemos a quedarnos con los 11 países seleccionados: España, Francia, Italia, Grecia, Turquía, Estados Unidos, China, India, México, Nueva Zelanda y Reino Unido. 

In [15]:
España = data2[data2["country"]=="Spain"]
Francia = data2[data2["country"]=="France"]
Italia = data2[data2["country"]=="Italy"]
Grecia = data2[data2["country"]=="Greece"]
Turquia = data2[data2["country"]=="Turkey"]
USA = data2[data2["country"]=="US"]
China = data2[data2["country"]=="China"]
India = data2[data2["country"]=="India"]
Mexico = data2[data2["country"]=="Mexico"]
Nueva_Zelanda = data2[data2["country"]=="New Zealand"]
Reino_Unido = data2[data2["country"]=="United Kingdom"]

paises = [España, Francia, Italia, Grecia, Turquia, USA, China, India, Mexico, Nueva_Zelanda, Reino_Unido]

In [16]:
data2 = pd.concat(paises)#Filtramos los países seleccionados
data2

,country,confirmed,deaths,recovered,active,%deaths,%recovered,%D/100R
157,Spain,272421,28432,150376,93613,10.44,55.20,18.91
61,France,220352,30212,81212,108928,13.71,36.86,37.20
85,Italy,246286,35112,198593,12581,14.26,80.64,17.68
67,Greece,4227,202,1374,2651,4.78,32.51,14.70
172,Turkey,227019,5630,210469,10920,2.48,92.71,2.67
173,US,4290259,148011,1325804,2816444,3.45,30.90,11.16
36,China,86783,4656,78869,3258,5.37,90.88,5.90
79,India,1480073,33408,951166,495499,2.26,64.26,3.51
111,Mexico,395489,44022,303810,47657,11.13,76.82,14.49
121,New Zealand,1557,22,1514,21,1.41,97.24,1.45


In [17]:
data2 = data2.reset_index(drop=True)#Reestructuramos el índice
data2

,country,confirmed,deaths,recovered,active,%deaths,%recovered,%D/100R
0,Spain,272421,28432,150376,93613,10.44,55.20,18.91
1,France,220352,30212,81212,108928,13.71,36.86,37.20
2,Italy,246286,35112,198593,12581,14.26,80.64,17.68
3,Greece,4227,202,1374,2651,4.78,32.51,14.70
4,Turkey,227019,5630,210469,10920,2.48,92.71,2.67
5,US,4290259,148011,1325804,2816444,3.45,30.90,11.16
6,China,86783,4656,78869,3258,5.37,90.88,5.90
7,India,1480073,33408,951166,495499,2.26,64.26,3.51
8,Mexico,395489,44022,303810,47657,11.13,76.82,14.49
9,New Zealand,1557,22,1514,21,1.41,97.24,1.45


## Exportación del dataset final 🚀

In [18]:
covid_data = data2
covid_data.to_csv("./covid_data.csv") #Exportación del DataFrame